In [ ]:
# import tensorflow as tf
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
import os
import numpy as np
import matplotlib.pylab as plt
from tensorflow.keras import (
    models,
    layers,
    optimizers,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
train_path = os.path.join(".", "train")
train_img_path = os.path.join(train_path, "imgs")

In [ ]:
def get_images_arr(path):
    return np.load(os.path.join(path, "X_imgs.npy")) / 255 * 0.2

In [ ]:
def get_data(path):
    return np.load(os.path.join(path, "X_curs.npy")) / 10, np.load(os.path.join(path, "y.npy")) / 10

In [ ]:
X_img = get_images_arr(train_path)
X_img_train, X_img_test = train_test_split(X_img, shuffle=False)

X_curs, y_curs = get_data(train_path)
X_curs_train, X_curs_test, y_train, y_test = train_test_split(X_curs, y_curs, shuffle=False)

In [ ]:
open_v = 0
max_v = 1
min_v = 2
close_v = 3

In [ ]:
y_train = np.array([[el[open_v]] for el in y_train])
y_test = np.array([[el[open_v]] for el in y_test])

In [ ]:
print(y_train.shape)
print(y_test.shape)

In [ ]:
x_scaler = StandardScaler()
x_scaler.fit(X_curs_train.flatten().reshape(-1, 1))
X_curs_train = x_scaler.transform(X_curs_train.flatten().reshape(-1, 1)).reshape(X_curs_train.shape)
X_curs_test = x_scaler.transform(X_curs_test.flatten().reshape(-1, 1)).reshape(X_curs_test.shape)

In [ ]:
# y_scaler = MinMaxScaler()
# y_scaler.fit(y_train)
# y_train = y_scaler.transform(y_train)
# y_test = y_scaler.transform(y_test)

In [ ]:
w = 144
h = 216
hours = 72
val = 4
n = 8
drop_value = 0.3
input_img = layers.Input(shape=(w, h))
img = layers.Reshape((w, h, 1))(input_img)
x1 = img
x1 = layers.BatchNormalization()(x1)
x1 = layers.SpatialDropout2D(drop_value)(x1)
x1 = layers.Conv2D(4 * n, (2, 2), activation="tanh")(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.SpatialDropout2D(drop_value)(x1)
x1 = layers.Conv2D(4 * n, (8, 3), activation="tanh")(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.SpatialDropout2D(drop_value)(x1)
x1 = layers.Conv2D(4 * n, (4, 5), activation="tanh")(x1)
x1 = layers.MaxPool2D()(x1)
x1_output = layers.GlobalMaxPooling2D()(x1)

x2 = x1
x2 = layers.BatchNormalization()(x2)
x2 = layers.SpatialDropout2D(drop_value)(x2)
x2 = layers.Conv2D(8 * n, (3, 1), activation="tanh")(x2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.SpatialDropout2D(drop_value)(x2)
x2 = layers.Conv2D(8 * n, (6, 3), activation="tanh")(x2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.SpatialDropout2D(drop_value)(x2)
x2 = layers.Conv2D(8 * n, (9, 5), activation="tanh")(x2)
x2 = layers.MaxPool2D()(x2)
x2_output = layers.GlobalMaxPooling2D()(x2)

x3 = x2
x3 = layers.BatchNormalization()(x3)
x3 = layers.SpatialDropout2D(drop_value)(x3)
x3 = layers.Conv2D(16 * n, (2, 2), activation="tanh")(x3)
x3 = layers.BatchNormalization()(x3)
x3 = layers.SpatialDropout2D(drop_value)(x3)
x3 = layers.Conv2D(16 * n, (7, 4), activation="tanh")(x3)
x3 = layers.BatchNormalization()(x3)
x3 = layers.SpatialDropout2D(drop_value)(x3)
x3 = layers.Conv2D(16 * n, (12, 7), activation="tanh")(x3)
x3 = layers.MaxPool2D()(x3)
x3_output = layers.GlobalMaxPooling2D()(x3)

x = layers.concatenate([x3_output, x2_output, x1_output])
x = layers.Activation("relu")(x)

input_val = layers.Input(shape=(hours, val))

y1 = input_val
y1 = layers.BatchNormalization()(y1)
y1 = layers.LSTM(16 * n, return_sequences=True)(y1)
y1 = layers.LSTM(8 * n, return_sequences=True)(y1)
y1 = layers.LSTM(4 * n, return_sequences=True)(y1)
y1 = layers.GlobalMaxPooling1D()(y1)

y2 = input_val
y2 = layers.BatchNormalization()(y2)
y2 = layers.LSTM(16 * n, return_sequences=True)(y2)
y2 = layers.LSTM(8 * n, return_sequences=True)(y2)
y2 = layers.LSTM(4 * n, return_sequences=False)(y2)

y = layers.concatenate([y1, y2])
z = layers.concatenate([x, y])

z = layers.BatchNormalization()(z)
z = layers.Dropout(drop_value)(z)
z = layers.Dense(8 * n, activation="relu")(z)
z = layers.BatchNormalization()(z)
z = layers.Dropout(drop_value)(z)
z = layers.Dense(16 * n, activation="relu")(z)
z = layers.BatchNormalization()(z)
z = layers.Dropout(drop_value)(z)
z = layers.Dense(32 * n, activation="relu")(z)
z = layers.BatchNormalization()(z)
z = layers.Dropout(drop_value)(z)
z = layers.Dense(16 * n, activation="relu")(z)
z = layers.BatchNormalization()(z)
z = layers.Dropout(drop_value)(z)
z = layers.Dense(8 * n, activation="softplus")(z)
output_val = layers.Dense(1, activation="linear")(z)

model = models.Model([input_img, input_val], output_val)

model.summary()

In [ ]:
model.compile(loss="mae", optimizer=optimizers.Adam(0.0001), metrics=["mse"])

utils.plot_model(model, show_shapes=True)

In [ ]:
def draw_history(history, key):
    plt.figure(figsize=(10, 10))
    plt.plot(history.history[key], label="Ошибка на обучающей выборке.")
    plt.plot(history.history[f"val_{key}"], label="Ошибка на проверочной выборке.")
    plt.xlabel("Эпоха обучения.")
    plt.ylabel("Ошибка.")
    plt.legend()
    plt.show()

history = model.fit(
    [
        X_img_train,
        X_curs_train,
    ],
    y_train,
    epochs=140,
    batch_size=64,
    verbose=True,
    validation_data=(
        [
            X_img_test,
            X_curs_test,
        ],
        y_test
    )
)

In [ ]:
# model.load_weights(os.path.join(".", "model_open.h5"))

model.save_weights(os.path.join(".", "model_open.h5"))

draw_history(history, "mse")
draw_history(history, "loss")

In [ ]:
# check_predict = model.predict(
#     [X_img_check[1:], X_curs_check[1:]]
# )
# print(check_predict[0])
# print(check_predict[1])

In [ ]:
# check_predict = y_scaler.inverse_transform(check_predict)

In [ ]:
# y_check *= 10
# check_predict *= 10

In [ ]:
# plt.figure(figsize=(30, 30))
# plt.plot(check_predict - 0.015, label="predict")
# plt.plot(y_check[1:, open_v], label="real", color="red")
# plt.legend()
# plt.show()